In [11]:
import numpy as np
import numpy.linalg as LA
import jax
import jax.numpy as jnp
import jax.numpy.linalg as JLA

import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
from jax.example_libraries import optimizers
from tqdm.notebook import trange
from functools import partial

In [2]:
T = 4.0
N = 200
eta = T/N
std = 0.1
train_itr = 500
m = 16
init_std = 1e-3

In [10]:
def gen_dataset(std, func):
    x_data = jnp.zeros(N)
    for i in range(N):
        if i % 10 == 0:
            x_data[i] = func(eta*i) + std*jnp.array(np.random.randn())
        else:
            x_data[i] = jnp.nan
    return x_data

In [12]:
def Euler(max_itr, W1, b1, W2, b2):
    x = jnp.zeros(2)
    for i in range(max_itr):
        x += eta*(W2*jax.nn.relu(W1*x+b1)+b2)
    return x[0]